In [4]:
import random
from collections import deque

weather_states = ["rain", "snow", "clear", "humid"]

# ----------------------------------------
# Initial Temperature
# ----------------------------------------

def select_initial_temperature(weather):
    if weather == "rain":
        return "mild"
    elif weather == "snow":
        return "cold"
    elif weather == "humid":
        return "hot"
    elif weather == "clear":
        return random.choice(["hot", "warm", "mild"])

# ----------------------------------------
# Temperature Progression
# ----------------------------------------

def temperature_progression(initial_temp):
    if initial_temp == "hot":
        return ["hot", "warm", "mild"]
    elif initial_temp == "warm":
        return ["warm", "mild"]
    elif initial_temp == "mild":
        return ["mild"]
    else:
        return [initial_temp]

# ----------------------------------------
# Deliberate Agent with Full Plan Display
# ----------------------------------------

class DeliberateSprinklerAgent:

    def __init__(self, weather, initial_temp):
        self.weather = weather
        self.temp_cycles = temperature_progression(initial_temp)

    def transition(self, state, action):

        cycle_index, soil = state
        temp = self.temp_cycles[cycle_index]

        if action > 0:
            soil = "wet"

        # Evaporation between cycles
        if cycle_index < len(self.temp_cycles) - 1:
            if temp in ["hot", "warm"]:
                soil = "dry"

        return (cycle_index + 1, soil)

    def simulate_plan(self, plan):

        state = (0, "dry")
        total_cost = 0

        for action in plan:
            state = self.transition(state, action)
            total_cost += action

        return state, total_cost

    def generate_all_plans(self):

        if self.weather in ["rain", "snow"]:
            return []

        queue = deque()
        queue.append((0, []))
        all_plans = []

        while queue:
            depth, path = queue.popleft()

            if depth == len(self.temp_cycles):
                final_state, cost = self.simulate_plan(path)
                all_plans.append((path, final_state, cost))
                continue

            for action in [0, 10, 20]:
                queue.append((depth + 1, path + [action]))

        return all_plans

    def select_best_plan(self, all_plans):

        best_plan = None
        best_cost = float('inf')

        for plan, final_state, cost in all_plans:
            cycle_index, soil = final_state

            if soil == "wet" and cost < best_cost:
                best_cost = cost
                best_plan = plan

        return best_plan, best_cost


# ----------------------------------------
# Simulation Runner
# ----------------------------------------

def run_simulation():

    print("\nEnter weather for 5 days (rain/snow/clear/humid)\n")

    daily_inputs = []
    for i in range(5):
        w = input(f"Day {i+1} Weather: ").lower()
        while w not in weather_states:
            print("Invalid input.")
            w = input(f"Day {i+1} Weather: ").lower()
        daily_inputs.append(w)

    print("\n========== DELIBERATE AGENT (SHOW ALL PLANS) ==========\n")

    for day in range(5):

        weather = daily_inputs[day]
        initial_temp = select_initial_temperature(weather)

        print(f"=========== DAY {day+1} ===========")
        print(f"Weather     : {weather}")
        print(f"Initial Temp: {initial_temp}\n")

        if weather in ["rain", "snow"]:
            print("Rain/Snow detected → Soil remains wet. No irrigation needed.\n")
            print("---------------------------------------------\n")
            continue

        agent = DeliberateSprinklerAgent(weather, initial_temp)

        all_plans = agent.generate_all_plans()

        print("All Possible Plans Explored:\n")

        for plan, final_state, cost in all_plans:
            _, soil = final_state
            print(f"Plan: {plan} | Final Soil: {soil} | Total Irrigation: {cost} minutes")

        best_plan, best_cost = agent.select_best_plan(all_plans)

        print("\nSelected Optimal Plan:\n")
        print(f"Best Plan: {best_plan}")
        print(f"Minimum Irrigation Time: {best_cost} minutes\n")

        print("---------------------------------------------\n")


run_simulation()


Enter weather for 5 days (rain/snow/clear/humid)



Day 1 Weather:  snow
Day 2 Weather:  rain
Day 3 Weather:  clear
Day 4 Weather:  humid
Day 5 Weather:  clear



========== DELIBERATE AGENT (SHOW ALL PLANS) ==========

=========== DAY 1 ===========
Weather     : snow
Initial Temp: cold

Rain/Snow detected → Soil remains wet. No irrigation needed.

---------------------------------------------

=========== DAY 2 ===========
Weather     : rain
Initial Temp: mild

Rain/Snow detected → Soil remains wet. No irrigation needed.

---------------------------------------------

=========== DAY 3 ===========
Weather     : clear
Initial Temp: hot

All Possible Plans Explored:

Plan: [0, 0, 0] | Final Soil: dry | Total Irrigation: 0 minutes
Plan: [0, 0, 10] | Final Soil: wet | Total Irrigation: 10 minutes
Plan: [0, 0, 20] | Final Soil: wet | Total Irrigation: 20 minutes
Plan: [0, 10, 0] | Final Soil: dry | Total Irrigation: 10 minutes
Plan: [0, 10, 10] | Final Soil: wet | Total Irrigation: 20 minutes
Plan: [0, 10, 20] | Final Soil: wet | Total Irrigation: 30 minutes
Plan: [0, 20, 0] | Final Soil: dry | Total Irrigation: 20 minutes
Plan: [0, 20, 10] | Final